In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import math
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_absolute_percentage_error
import torch.optim as optim
import torch
import torch.nn as nn
from sklearn.metrics import mean_squared_error
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import datetime

In [ ]:
import os
import random
import sklearn

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    tf.random.set_seed(seed)


# seed_everything(0xAAAAAAA)
seed_everything(12345)

---------------------------------------------------------------
Without Polarity

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
stock_data = pd.read_csv("/content/gdrive/MyDrive/project/stock_vader.csv")

In [ ]:
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

In [ ]:
stock_data['Close'] = pd.to_numeric(stock_data['Close'], downcast="float")

In [ ]:
stock_data_info = stock_data[['Date', 'Close']]

In [ ]:
!pip install darts

--------------------------------------------------------------------------

In [ ]:
from darts.utils.timeseries_generation import datetime_attribute_timeseries

In [ ]:
history_dataset = stock_data_info
history_dataset = history_dataset[['Close', 'Date']]
history_dataset.reset_index(inplace=True)
history_dataset['Date'] = pd.to_datetime(pd.to_datetime(history_dataset['Date']).dt.date)
history_dataset['Date']

<ipython-input-462-2863482ebb9b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_dataset['Date'] = pd.to_datetime(pd.to_datetime(history_dataset['Date']).dt.date)


0     2017-01-03
1     2017-01-04
2     2017-01-05
3     2017-01-06
4     2017-01-09
         ...    
748   2019-12-23
749   2019-12-24
750   2019-12-26
751   2019-12-27
752   2019-12-30
Name: Date, Length: 753, dtype: datetime64[ns]

In [ ]:
from darts import TimeSeries
stock_series = TimeSeries.from_dataframe(history_dataset, time_col='Date', fill_missing_dates=True, freq='b')

In [ ]:
from darts.models import TFTModel

In [ ]:
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller

In [ ]:
filler = MissingValuesFiller()
scaler = Scaler()
stock_series_scale = scaler.fit_transform(
    filler.transform(stock_series)).astype(np.float32)

In [ ]:
transformer = Scaler()

stock_series_scale = transformer.fit_transform(stock_series_scale)

In [ ]:
covariates = datetime_attribute_timeseries(stock_series_scale, attribute="year", one_hot=False)
covariates = covariates.stack(
    datetime_attribute_timeseries(stock_series_scale, attribute="month", one_hot=False)
)
covariates = covariates.stack(
  TimeSeries.from_times_and_values(
      times=stock_series_scale.time_index,
      values=np.arange(len(stock_series_scale)),
      columns=["linear_increase"],
  )
)

covariates = covariates.astype(np.float32)
scaler_covs = Scaler()
covariates_transformed = scaler_covs.fit_transform(covariates)

In [ ]:
my_model = TFTModel(8, 1, n_epochs=25, random_state=1234)

In [ ]:
train_len = int(len(stock_series_scale) * 0.8)

In [ ]:
base_train, val = stock_series_scale[:-train_len], stock_series_scale[-train_len:]

In [ ]:
my_model.fit(base_train, future_covariates=covariates_transformed, verbose=True)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
   | Name                              | Type                             | Params
----------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0     
1  | val_metrics                       | MetricCollection                 | 0     
2  | input_embeddings                  | _MultiEmbedding                  | 0     
3  | static_covariates_vsn             | _VariableSelectionNetwork        | 0     
4  | encoder_vsn                       | _VariableSelectionNetwork        | 3.0 K 
5  | decoder_vsn                       | _Vari

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=25` reached.


TFTModel(input_chunk_length=8, output_chunk_length=1, hidden_size=16, lstm_layers=1, num_attention_heads=4, full_attention=False, feed_forward=GatedResidualNetwork, dropout=0.1, hidden_continuous_size=8, categorical_embedding_sizes=None, add_relative_index=False, loss_fn=None, likelihood=None, norm_type=LayerNorm, use_static_covariates=True, n_epochs=25, random_state=1234)

In [ ]:
base_prediction = my_model.historical_forecasts(stock_series_scale, future_covariates=covariates_transformed, start = val.time_index[0], forecast_horizon = 3, stride = 3, retrain = False, last_points_only = False)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:

In [ ]:
def conc_pred(prediction):
  pred = prediction[0]
  for i in range(1, len(prediction)):
    pred = pred.concatenate(prediction[i])

  return pred

In [ ]:
base_prediction = transformer.inverse_transform(conc_pred(base_prediction))

In [ ]:
from darts.metrics import rmse, mape

In [ ]:
print(rmse(val, base_prediction))

0.4944526


In [ ]:
print(mape(val, base_prediction))

71.3312417268753


---------------------------------------------------------------------------
Vader Polarity

In [ ]:
stock_data = pd.read_csv("/content/gdrive/MyDrive/project/stock_vader.csv")

In [ ]:
stock_data['Polarity'] = pd.to_numeric(stock_data['Polarity'], downcast="float")

In [ ]:
stock_data['sum'] = pd.to_numeric(stock_data['sum'], downcast="float")

In [ ]:
stock_data["Polarity"] = stock_data["Polarity"] / stock_data["sum"]

In [ ]:
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

In [ ]:
stock_data['Close'] = pd.to_numeric(stock_data['Close'], downcast="float")

In [ ]:
stock_data_info = stock_data[['Date', 'Close']]

In [ ]:
history_dataset = stock_data_info
history_dataset = history_dataset[['Close', 'Date']]
history_dataset.reset_index(inplace=True)
history_dataset['Date'] = pd.to_datetime(pd.to_datetime(history_dataset['Date']).dt.date)

<ipython-input-494-dd325024af0c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_dataset['Date'] = pd.to_datetime(pd.to_datetime(history_dataset['Date']).dt.date)


In [ ]:
from darts import TimeSeries
stock_series = TimeSeries.from_dataframe(history_dataset, time_col='Date', fill_missing_dates=True, freq='b')

In [ ]:
from darts.models import TFTModel

In [ ]:
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller

In [ ]:
filler = MissingValuesFiller()
scaler = Scaler()
stock_series_scale = scaler.fit_transform(
    filler.transform(stock_series)).astype(np.float32)

In [ ]:
transformer = Scaler()

stock_series_scale = transformer.fit_transform(stock_series_scale)

In [ ]:
covariates = datetime_attribute_timeseries(stock_series_scale, attribute="year", one_hot=False)
covariates = covariates.stack(
    datetime_attribute_timeseries(stock_series_scale, attribute="month", one_hot=False)
)
covariates = covariates.stack(
  TimeSeries.from_times_and_values(
      times=stock_series_scale.time_index,
      values=np.arange(len(stock_series_scale)),
      columns=["linear_increase"],
  )
)

covariates = covariates.astype(np.float32)
scaler_covs = Scaler()
covariates_transformed = scaler_covs.fit_transform(covariates)

In [ ]:
polarity_data = stock_data[['Date', 'Polarity']]

In [ ]:
polarity_series = TimeSeries.from_dataframe(polarity_data, time_col='Date', fill_missing_dates=True, freq='b')

In [ ]:
polarity_series = scaler.fit_transform(
    filler.transform(polarity_series)).astype(np.float32)

In [ ]:
my_model = TFTModel(8, 1, n_epochs=25, random_state=1234)

In [ ]:
train_len = int(len(stock_series_scale) * 0.8)

In [ ]:
base_train, val = stock_series_scale[:-train_len], stock_series_scale[-train_len:]

In [ ]:
my_model.fit(base_train, future_covariates=covariates_transformed, past_covariates=polarity_series, verbose=True)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
   | Name                              | Type                             | Params
----------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0     
1  | val_metrics                       | MetricCollection                 | 0     
2  | input_embeddings                  | _MultiEmbedding                  | 0     
3  | static_covariates_vsn             | _VariableSelectionNetwork        | 0     
4  | encoder_vsn                       | _VariableSelectionNetwork        | 3.7 K 
5  | decoder_vsn                       | _Vari

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=25` reached.


TFTModel(input_chunk_length=8, output_chunk_length=1, hidden_size=16, lstm_layers=1, num_attention_heads=4, full_attention=False, feed_forward=GatedResidualNetwork, dropout=0.1, hidden_continuous_size=8, categorical_embedding_sizes=None, add_relative_index=False, loss_fn=None, likelihood=None, norm_type=LayerNorm, use_static_covariates=True, n_epochs=25, random_state=1234)

In [ ]:
base_prediction = my_model.historical_forecasts(stock_series_scale, future_covariates=covariates_transformed, past_covariates=polarity_series, start = val.time_index[0], forecast_horizon = 3, stride = 3, retrain = False, last_points_only = False)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:

In [ ]:
def conc_pred(prediction):
  pred = prediction[0]
  for i in range(1, len(prediction)):
    pred = pred.concatenate(prediction[i])

  return pred

In [ ]:
base_prediction = transformer.inverse_transform(conc_pred(base_prediction))

In [ ]:
from darts.metrics import rmse, mape

In [ ]:
print(rmse(val, base_prediction))

0.48952842


In [ ]:
print(mape(val, base_prediction))

65.33742547035217


---------------------------------------------------------------------------
Random polarity

In [ ]:
stock_data = pd.read_csv("/content/gdrive/MyDrive/project/stock_vader.csv")

In [ ]:
stock_data["Polarity"] = stock_data["Polarity"].apply(lambda x: np.random.random() * 2 - 1)

In [ ]:
stock_data['Polarity'] = pd.to_numeric(stock_data['Polarity'], downcast="float")

In [ ]:
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

In [ ]:
stock_data['Close'] = pd.to_numeric(stock_data['Close'], downcast="float")

In [ ]:
stock_data_info = stock_data[['Date', 'Close']]

In [ ]:
history_dataset = stock_data_info
history_dataset = history_dataset[['Close', 'Date']]
history_dataset.reset_index(inplace=True)
history_dataset['Date'] = pd.to_datetime(pd.to_datetime(history_dataset['Date']).dt.date)

<ipython-input-521-dd325024af0c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_dataset['Date'] = pd.to_datetime(pd.to_datetime(history_dataset['Date']).dt.date)


In [ ]:
from darts import TimeSeries
stock_series = TimeSeries.from_dataframe(history_dataset, time_col='Date', fill_missing_dates=True, freq='b')

In [ ]:
from darts.models import TFTModel

In [ ]:
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller

In [ ]:
filler = MissingValuesFiller()
scaler = Scaler()
stock_series_scale = scaler.fit_transform(
    filler.transform(stock_series)).astype(np.float32)

In [ ]:
transformer = Scaler()

stock_series_scale = transformer.fit_transform(stock_series_scale)

In [ ]:
covariates = datetime_attribute_timeseries(stock_series_scale, attribute="year", one_hot=False)
covariates = covariates.stack(
    datetime_attribute_timeseries(stock_series_scale, attribute="month", one_hot=False)
)
covariates = covariates.stack(
  TimeSeries.from_times_and_values(
      times=stock_series_scale.time_index,
      values=np.arange(len(stock_series_scale)),
      columns=["linear_increase"],
  )
)

covariates = covariates.astype(np.float32)
scaler_covs = Scaler()
covariates_transformed = scaler_covs.fit_transform(covariates)

In [ ]:
polarity_data = stock_data[['Date', 'Polarity']]

In [ ]:
polarity_series = TimeSeries.from_dataframe(polarity_data, time_col='Date', fill_missing_dates=True, freq='b')

In [ ]:
polarity_series = scaler.fit_transform(
    filler.transform(polarity_series)).astype(np.float32)

In [ ]:
my_model = TFTModel(8, 1, n_epochs=25, random_state=1234)

In [ ]:
train_len = int(len(stock_series_scale) * 0.8)

In [ ]:
base_train, val = stock_series_scale[:-train_len], stock_series_scale[-train_len:]

In [ ]:
my_model.fit(base_train, future_covariates=covariates_transformed, past_covariates=polarity_series, verbose=True)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
   | Name                              | Type                             | Params
----------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0     
1  | val_metrics                       | MetricCollection                 | 0     
2  | input_embeddings                  | _MultiEmbedding                  | 0     
3  | static_covariates_vsn             | _VariableSelectionNetwork        | 0     
4  | encoder_vsn                       | _VariableSelectionNetwork        | 3.7 K 
5  | decoder_vsn                       | _Vari

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=25` reached.


TFTModel(input_chunk_length=8, output_chunk_length=1, hidden_size=16, lstm_layers=1, num_attention_heads=4, full_attention=False, feed_forward=GatedResidualNetwork, dropout=0.1, hidden_continuous_size=8, categorical_embedding_sizes=None, add_relative_index=False, loss_fn=None, likelihood=None, norm_type=LayerNorm, use_static_covariates=True, n_epochs=25, random_state=1234)

In [ ]:
base_prediction = my_model.historical_forecasts(stock_series_scale, future_covariates=covariates_transformed, past_covariates=polarity_series, start = val.time_index[0], forecast_horizon = 3, stride = 3, retrain = False, last_points_only = False)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:

In [ ]:
def conc_pred(prediction):
  pred = prediction[0]
  for i in range(1, len(prediction)):
    pred = pred.concatenate(prediction[i])

  return pred

In [ ]:
base_prediction = transformer.inverse_transform(conc_pred(base_prediction))

In [ ]:
from darts.metrics import rmse, mape

In [ ]:
print(rmse(val, base_prediction))

0.48873925


In [ ]:
print(mape(val, base_prediction))

65.28637111186981


---------------------------------------------------------------------------
Polarity with fin_roberta

In [ ]:
stock_data = pd.read_csv("/content/gdrive/MyDrive/project/fin_roberta_stock_data0.csv")

In [ ]:
stock_data['Polarity'] = pd.to_numeric(stock_data['Polarity'], downcast="float")

In [ ]:
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

In [ ]:
stock_data['Close'] = pd.to_numeric(stock_data['Close'], downcast="float")

In [ ]:
stock_data_info = stock_data[['Date', 'Close']]

In [ ]:
history_dataset = stock_data_info
history_dataset = history_dataset[['Close', 'Date']]
history_dataset.reset_index(inplace=True)
history_dataset['Date'] = pd.to_datetime(pd.to_datetime(history_dataset['Date']).dt.date)

<ipython-input-554-dd325024af0c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_dataset['Date'] = pd.to_datetime(pd.to_datetime(history_dataset['Date']).dt.date)


In [ ]:
from darts import TimeSeries
stock_series = TimeSeries.from_dataframe(history_dataset, time_col='Date', fill_missing_dates=True, freq='b')

In [ ]:
from darts.models import TFTModel

In [ ]:
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller

In [ ]:
filler = MissingValuesFiller()
scaler = Scaler()
stock_series_scale = scaler.fit_transform(
    filler.transform(stock_series)).astype(np.float32)

In [ ]:
transformer = Scaler()

stock_series_scale = transformer.fit_transform(stock_series_scale)

In [ ]:
covariates = datetime_attribute_timeseries(stock_series_scale, attribute="year", one_hot=False)
covariates = covariates.stack(
    datetime_attribute_timeseries(stock_series_scale, attribute="month", one_hot=False)
)
covariates = covariates.stack(
  TimeSeries.from_times_and_values(
      times=stock_series_scale.time_index,
      values=np.arange(len(stock_series_scale)),
      columns=["linear_increase"],
  )
)

covariates = covariates.astype(np.float32)
scaler_covs = Scaler()
covariates_transformed = scaler_covs.fit_transform(covariates)

In [ ]:
polarity_data = stock_data[['Date', 'Polarity']]

In [ ]:
polarity_series = TimeSeries.from_dataframe(polarity_data, time_col='Date', fill_missing_dates=True, freq='b')

In [ ]:
polarity_series = scaler.fit_transform(
    filler.transform(polarity_series)).astype(np.float32)

In [ ]:
my_model = TFTModel(8, 1, n_epochs=25, random_state=1234)

In [ ]:
train_len = int(len(stock_series_scale) * 0.8)

In [ ]:
base_train, val = stock_series_scale[:-train_len], stock_series_scale[-train_len:]

In [ ]:
my_model.fit(base_train, future_covariates=covariates_transformed, past_covariates=polarity_series, verbose=True)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
   | Name                              | Type                             | Params
----------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0     
1  | val_metrics                       | MetricCollection                 | 0     
2  | input_embeddings                  | _MultiEmbedding                  | 0     
3  | static_covariates_vsn             | _VariableSelectionNetwork        | 0     
4  | encoder_vsn                       | _VariableSelectionNetwork        | 3.7 K 
5  | decoder_vsn                       | _Vari

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=25` reached.


TFTModel(input_chunk_length=8, output_chunk_length=1, hidden_size=16, lstm_layers=1, num_attention_heads=4, full_attention=False, feed_forward=GatedResidualNetwork, dropout=0.1, hidden_continuous_size=8, categorical_embedding_sizes=None, add_relative_index=False, loss_fn=None, likelihood=None, norm_type=LayerNorm, use_static_covariates=True, n_epochs=25, random_state=1234)

In [ ]:
base_prediction = my_model.historical_forecasts(stock_series_scale, future_covariates=covariates_transformed, past_covariates=polarity_series, start = val.time_index[0], forecast_horizon = 3, stride = 3, retrain = False, last_points_only = False)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:

In [ ]:
def conc_pred(prediction):
  pred = prediction[0]
  for i in range(1, len(prediction)):
    pred = pred.concatenate(prediction[i])

  return pred

In [ ]:
base_prediction = transformer.inverse_transform(conc_pred(base_prediction))

In [ ]:
from darts.metrics import rmse, mape

In [ ]:
print(rmse(val, base_prediction))

0.48908243


In [ ]:
print(mape(val, base_prediction))

65.36707282066345


-------------------------------------------------------
With Embeddings

In [ ]:
stock_data = pd.read_csv("/content/gdrive/MyDrive/project/fin_roberta_stock_data_emb_data_withstock_0.csv")

In [ ]:
stock_data['Date'] = stock_data['Unnamed: 0']

In [ ]:
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

In [ ]:
stock_data['Close'] = pd.to_numeric(stock_data['Close'], downcast="float")

In [ ]:
stock_data_info = stock_data[['Date', 'Close']]

In [ ]:
history_dataset = stock_data_info
history_dataset = history_dataset[['Close', 'Date']]
history_dataset.reset_index(inplace=True)
history_dataset['Date'] = pd.to_datetime(pd.to_datetime(history_dataset['Date']).dt.date)

<ipython-input-579-dd325024af0c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_dataset['Date'] = pd.to_datetime(pd.to_datetime(history_dataset['Date']).dt.date)


In [ ]:
from darts import TimeSeries
stock_series = TimeSeries.from_dataframe(history_dataset, time_col='Date', fill_missing_dates=True, freq='b')

In [ ]:
from darts.models import TFTModel

In [ ]:
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller

In [ ]:
filler = MissingValuesFiller()
scaler = Scaler()
stock_series_scale = scaler.fit_transform(
    filler.transform(stock_series)).astype(np.float32)

In [ ]:
transformer = Scaler()

stock_series_scale = transformer.fit_transform(stock_series_scale)

In [ ]:
covariates = datetime_attribute_timeseries(stock_series_scale, attribute="year", one_hot=False)
covariates = covariates.stack(
    datetime_attribute_timeseries(stock_series_scale, attribute="month", one_hot=False)
)
covariates = covariates.stack(
  TimeSeries.from_times_and_values(
      times=stock_series_scale.time_index,
      values=np.arange(len(stock_series_scale)),
      columns=["linear_increase"],
  )
)

covariates = covariates.astype(np.float32)
scaler_covs = Scaler()
covariates_transformed = scaler_covs.fit_transform(covariates)

In [ ]:
emb_data = stock_data[np.append('Date', np.array(range(1, 769)).astype(str))] 

In [ ]:
emb_series = TimeSeries.from_dataframe(emb_data, time_col='Date', fill_missing_dates=True, freq='b')

In [ ]:
emb_series = scaler.fit_transform(
    filler.transform(emb_series)).astype(np.float32)

In [ ]:
my_model = TFTModel(8, 1, n_epochs=25, random_state=1234)

In [ ]:
train_len = int(len(stock_series_scale) * 0.8)

In [ ]:
base_train, val = stock_series_scale[:-train_len], stock_series_scale[-train_len:]

In [ ]:
my_model.fit(base_train, future_covariates=covariates_transformed, past_covariates=emb_series, verbose=True)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
   | Name                              | Type                             | Params
----------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0     
1  | val_metrics                       | MetricCollection                 | 0     
2  | input_embeddings                  | _MultiEmbedding                  | 0     
3  | static_covariates_vsn             | _VariableSelectionNetwork        | 0     
4  | encoder_vsn                       | _VariableSelectionNetwork        | 537 K 
5  | decoder_vsn                       | _Vari

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=25` reached.


TFTModel(input_chunk_length=8, output_chunk_length=1, hidden_size=16, lstm_layers=1, num_attention_heads=4, full_attention=False, feed_forward=GatedResidualNetwork, dropout=0.1, hidden_continuous_size=8, categorical_embedding_sizes=None, add_relative_index=False, loss_fn=None, likelihood=None, norm_type=LayerNorm, use_static_covariates=True, n_epochs=25, random_state=1234)

In [ ]:
base_prediction = my_model.historical_forecasts(stock_series_scale, future_covariates=covariates_transformed, past_covariates=emb_series, start = val.time_index[0], forecast_horizon = 3, stride = 3, retrain = False, last_points_only = False)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:

In [ ]:
def conc_pred(prediction):
  pred = prediction[0]
  for i in range(1, len(prediction)):
    pred = pred.concatenate(prediction[i])

  return pred

In [ ]:
base_prediction = transformer.inverse_transform(conc_pred(base_prediction))

In [ ]:
from darts.metrics import rmse, mape

In [ ]:
print(rmse(val, base_prediction))

0.518579


In [ ]:
print(mape(val, base_prediction))

72.77274429798126
